In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pygooglenews import GoogleNews
from newspaper import Article
from newspaper import Config
from textblob import TextBlob
import csv

In [2]:
nltk.download('vader_lexicon')
nltk.download('punkt')

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Update vader lexicon with LughornMcDonald Finance lexicon and stock lexicon

# wordList = pd.read_csv('LoughranMcDonald_MasterDictionary_2020.csv')
# LM_negative = wordList['Word'][wordList['Negative']>2000]
# LM_positive = wordList['Word'][wordList['Positive']>2000]
# LM_negative.to_csv('LM_negative.csv',index=False)
# LM_positive.to_csv('LM_positive.csv',index=False)

sia = SentimentIntensityAnalyzer()

# stock market lexicon
stock_lex = pd.read_csv('Resources/lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# Loughran and McDonald
positive = []
with open('Resources/lexicon_data/LM_positive.csv', 'r') as f:
    reader = csv.reader(f)
    # Skip the header
    next(reader)
    for row in reader:
        positive.append(row[0].strip())
  
negative = []
with open('Resources/lexicon_data/LM_negative.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [4]:
# Function to get news headlines about 'search' on 'date'
def get_news(search, date):
    stories = []
    # timedelta for no. of days from 'date'
    delta = dt.timedelta(days = 1)
    
    # Instance of GoogleNews class in pygooglenews package
    gn = GoogleNews()
    # Search google for the news headlines
    result = gn.search(search, from_ = date.strftime('%m-%d-%Y'), to_ = (date+delta).strftime('%m-%d-%Y'))
    
    
    newsitem = result['entries']
    
    for item in newsitem:
        story = {
            'date':date,
            'title': item.title,
            'link': item.link,
            'published':item.published
        }
        stories.append(story)
    
    # Return all the headlines retrieved
    return stories
    
    

In [5]:
def get_gnews_article(df):
    list = []
    for i in range(0, df.shape[0]):
        dict = {}
        article = Article(df['link'][i], config = config)
        try:
            article.download()
            article.parse()
            article.nlp()       
            dict['Date'] = df['date'][i]
            dict['Title']=article.title
            dict['Article']=article.text
            dict['Summary'] = article.summary
            dict['Key_words'] = article.keywords
        except:
            pass
        
        list.append(dict)
        
    check_empty = not any(list)
        
    if check_empty == False:
        news_df = pd.DataFrame(list)
    else:
        news_df = pd.DataFrame()
    
    return news_df   

In [6]:
def calculate_sim_score(d1, d2):
    set_a, set_b = set(d1), set(d2)
    return len(set_a and set_b) / len(set_a or set_b)

def get_sentiments(news_df):
       
    news_df_row = news_df.shape[0]
    news_df_col = news_df.shape[1]
    
    # Empty lists to append sentiment calculation
    news_sentiment_subjectivity_list = []
    news_sentiment_vader_comp_list = []

     #load the descriptions into textblob
    for i in range(news_df_row):
        desc_blob = [TextBlob(desc) for desc in (news_df.iloc[i, 1:news_df_col])]
            
        #news_sentiment['tb_Pol'] = [b.sentiment.polarity for b in desc_blob] 
        
        # Get subjectivity score for news_df
        news_sentiment_subjectivity_list.append([b.sentiment.subjectivity for b in desc_blob])
        
        # Get compound score from vader analysis
        news_sentiment_vader_comp_list.append([sia.polarity_scores(v)['compound'] for v in news_df.iloc[i, 1:news_df_col]])
    
#     news_sentiment['vader_pos'] = [sia.polarity_scores(v)['pos'] for v in news_df.iloc[i, 1:news_df_col]]
#     news_sentiment['vader_neu'] = [sia.polarity_scores(v)['neu'] for v in news_df.iloc[i, 1:news_df_col]]
#     news_sentiment['vader_neg'] = [sia.polarity_scores(v)['neg'] for v in news_df.iloc[i, 1:news_df_col]]
    
#     news_sentiment_list.append(news_sentiment)
# news_sentiment_df = pd.concat(news_sentiment_list)
# news_sentiment_df

    # Create dataframes from the generated lists
    news_subjectivity_df = pd.DataFrame(news_sentiment_subjectivity_list)
    news_vader_compound_df = pd.DataFrame(news_sentiment_vader_comp_list)
    
   
    news_sim_score = []

    #Calculate similarity score for weighing the sentiment score
    for i in range(news_df_row):
        sim_score_list = []
        for j in range(0, news_df_col-1):
            sim_scores = []
            for k in range(0,(news_df_col-1)):
                if j != k:
                    sim_scores.append(calculate_sim_score(news_df.iloc[i][j], news_df.iloc[i][k]))
            sim_score_list.append(np.mean(sim_scores))
        news_sim_score.append(sim_score_list)
    
    # Create the similarity score dataFrame    
    news_sim_score_df = pd.DataFrame(news_sim_score)
    
    # Sentiment score generated based on subjectivity score
    news_weighted_subj_senti_df = news_subjectivity_df * news_vader_compound_df
    
    #Sentiment score generated based on similarity score
    news_weighted_simi_senti_df = news_sim_score_df * news_vader_compound_df
    
    # Generate the sentiment dataframe
    news_sentiment_df = []
    news_sentiment_df = news_df[['index']]
    news_sentiment_df['subj_score'] = news_weighted_subj_senti_df.mean(axis = 1, skipna = True)
    news_sentiment_df['simi_score'] = news_weighted_simi_senti_df.mean(axis = 1, skipna = True)
    news_sentiment_df['vader_score'] = news_vader_compound_df.mean(axis = 1, skipna = True)
    return news_sentiment_df

In [7]:
%%time
# save the user defined search term in a variable
key = input("Please provide the search term: ")

#Input the number of days for which we want the google news for
no_days = input(" Enter the number of days for which you want the news headlines:")

# A list of dates from today going back 'no_days' in ascending order
datelist = pd.bdate_range(end = dt.date.today(), periods = int(no_days))

summary_df = []

# iterate through the list of dates to get the headlines for each date
for date in datelist:
    df = pd.DataFrame(get_news(key, date))
    if(len(df) > 20):
        article_df = get_gnews_article(df[:20]) 
    else:
        article_df = get_gnews_article(df)
    summary_df.append(article_df)

Please provide the search term:  tsla
 Enter the number of days for which you want the news headlines: 5


C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\dateparser\date_parser.py:37: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


Wall time: 2min 26s


In [8]:
df2 = pd.concat(summary_df).set_index('Date')
df2 = df2.dropna()
df2 = df2.reset_index()
display(df2.head())
display(df2.tail())

,Date,Title,Article,Summary,Key_words
0,2022-01-10,"Why Tesla Stock First Crashed, Then Bounced Today",What happened\n\nTopsy-turvy Tesla (NASDAQ:TSL...,What happenedTopsy-turvy Tesla (NASDAQ:TSLA) s...,"[production, wedbush, company, goldman, bounce..."
1,2022-01-10,Solar Power and Battery Storage Will Be the Re...,Elon Musk has long insisted that Tesla Inc. is...,Elon Musk has long insisted that Tesla Inc. is...,"[power, real, battery, vehicles, think, storag..."
2,2022-01-10,Tesla’s Most Prominent Black Executive Steps D...,Tesla Inc.’s head of human resources and one o...,Tesla Inc.’s head of human resources and one o...,"[job, shows, steps, hr, black, prominent, exec..."
3,2022-01-10,Tesla price target raised at Goldman Sachs,Tesla (TSLA) stock closed 3% higher on Monday ...,Tesla (TSLA) stock closed 3% higher on Monday ...,"[twitter, raised, goldman, deliveries, tesla, ..."
4,2022-01-10,"EV Stocks: Why Are TSLA, RIVN, XPEV, RIDE, NKL...","EV Stocks: Why Are TSLA, RIVN, XPEV, RIDE, NKL...","EV Stocks: Why Are TSLA, RIVN, XPEV, RIDE, NKL...","[riding, lower, lcid, ride, tsla, stocks, nkla..."


,Date,Title,Article,Summary,Key_words
94,2022-01-14,"Stone House Investment Management, LLC Buys iS...",Investment company Stone House Investment Mana...,Investment company Stone House Investment Mana...,"[growth, tips, management, portfolio, house, i..."
95,2022-01-14,McLean Asset Management Corp Buys Avantis U.S....,Investment company McLean Asset Management Cor...,Investment company McLean Asset Management Cor...,"[asset, management, avantis, holding, shares, ..."
96,2022-01-14,Dock Street Asset Management Inc Buys Tesla In...,"Greenwich, CT, based Investment company Dock S...","Greenwich, CT, based Investment company Dock S...","[prices, asset, management, portfolio, holding..."
97,2022-01-14,"Dow Jones Sells Off; Citigroup, JPMorgan Tumbl...",The Dow Jones Industrial Average fell sharply ...,Dow Jones Today:After the stock market open Fr...,"[sells, earnings, ibd, point, jpmorgan, tumble..."
98,2022-01-14,Why Nio Stock Faces an Uphill Battle,,,"[battle, uphill, faces, nio, stock]"


In [9]:
filepath = 'Resources/news_data/{}_test.csv'.format(key)
df2.to_csv(filepath)

In [10]:
dic = {}
for i in range(len(df2)):
    date = df2['Date'][i]
    summary = df2['Article'][i]
    if date in dic:
        dic[date].append(summary)
    else:
        dic[date] = [summary]

news_df = pd.DataFrame.from_dict(dic, orient = 'index')
news_df = news_df.reset_index()

In [11]:
news_df['index'] = pd.to_datetime(news_df['index'])
news_df.head(8)

,index,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,2022-01-10,What happened\n\nTopsy-turvy Tesla (NASDAQ:TSL...,Elon Musk has long insisted that Tesla Inc. is...,Tesla Inc.’s head of human resources and one o...,Tesla (TSLA) stock closed 3% higher on Monday ...,"EV Stocks: Why Are TSLA, RIVN, XPEV, RIDE, NKL...","To ensure this doesn’t happen in the future, p...",An exchange-traded fund with holdings picked b...,"Tesla is on the verge of opening a new Austin,...",I am neutral on Tesla (NASDAQ: TSLA) as it has...,...,Tesla Daily Chart.\n\nThe trap move on the dai...,The Dow Jones Industrial average closed lower ...,US Stocks Elliott Wave Analysis: Amazon AMZN E...,By Sinéad Carew and Caroline Valetkevitch\n\nJ...,Pre-Open Stock Movers:\n\nZynga (NASDAQ: ZNGA)...,,RomoloTavani/iStock via Getty Images\n\nThesis...,,"LONDON, Jan 10 (Reuters) - Nickel prices held ...",None
1,2022-01-11,"Tesla, Inc.'s (TSLA) shares reversed course fr...",Tesla TSLA recently signed its first nickel su...,TLOFF Stock: The $1.5B Tesla Deal That Has Nic...,Tesla Inc. won approval to build more test car...,Cathie Wood has risen in the ranks of institut...,Tesla stock flips from red to green on Monday ...,The China Passenger Car Association reported b...,"Valerie Capers Workman, head of the company's ...",3 EV Stocks With Excellent Long-Term Potential...,...,Investment company Lynch & Associates (Current...,The logo of LG Chem is seen at its office buil...,Dogecoin Tesla Payments Are Rumored to be Comi...,The stock of Ford Motor Co. (F) - Get Ford Mot...,"Investment company First American Trust, Fsb (...",Updated at 4:15 pm U.S. EST\n\nStocks finished...,"Cathie Wood, the star fund manager and chief e...",There's a quick way to summarize the S&P 500 t...,The Dow Jones gained as Federal Reserve chairm...,. Upgrade your Investing.com experience. Save ...
2,2022-01-12,jetcityimage/iStock Editorial via Getty Images...,"Tesla (TSLA) is a ""barometer"" of market confid...",By\n\nTesla (NASDAQ:TSLA) will hold its Q4 and...,What happened\n\nShares of Tesla (NASDAQ:TSLA)...,jetcityimage/iStock Editorial via Getty Images...,Stay on top of the electric car revolution by ...,,Investors Should Expect Significantly Higher F...,Tesla (TSLA) strong rally this week points to ...,...,The logo of car manufacturer Tesla is seen at ...,A Tesla vehicle drives past Tesla's primary ve...,The day is starting out a little odd for Tesla...,The Dow Jones Industrial Average moved higher ...,"Comparisons can sometimes be odious, especiall...",Summary Eikon search string for individual sto...,Investment company Old Port Advisors (Current ...,"The ARKK ETF has had a stellar three-day run, ...",Cars for export wait to be loaded onto cargo v...,U.S. Federal Reserve Board Chairman Jerome Pow...
3,2022-01-13,What happened\n\nTesla (NASDAQ:TSLA) took anot...,"David Colombo, a 19-year-old cybersecurity res...",TSLA stock surges higher on Wednesday as risk ...,Tesla (TSLA) online shop now accepting Dogecoi...,Tesla electric vehicles have conquered the U.S...,"Recently, several Tesla (NASDAQ: TSLA) owners ...",The Dow Jones Industrial Average reversed lowe...,"That’s why they’re the best, they’re the posit...",Deutsche Bank analyst Emannual Rosner has weig...,...,The fourth-quarter earnings season is heating ...,"Summary Nasdaq, S&P down; Dow advances\n\nInfo...",NASDAQ:RIVN gained 3.51% during Wednesday’s tr...,Dow Jones futures rose slightly early Thursday...,"In this segment of ""3 Minute Stocks Updates"" o...",See the latest news for the top stocks in the ...,Summary All three major U.S. stock indexes low...,"InvestorPlace - Stock Market News, Stock Advic...","Summary Nasdaq, S&P down; Dow advances\n\nIndu...",A Tesla is charged at an electric car supercha...
4,2022-01-14,"Electric carmaker Tesla, Inc. (TSLA) may have ...",Tesla stock dumps on Thursday as tech takes a ...,Among the underlying components of the Russell...,"Tesla (TSLA) appears an attractive pick, as it...","Auto sa

In [12]:
news_df = news_df.replace(np.nan, 'none')
news_df = news_df.replace('', 'none')

sentiment_df = get_sentiments(news_df)

sentiment_df

C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,index,subj_score,simi_score,vader_score
0,2022-01-10,0.284667,0.465190,0.647135
1,2022-01-11,0.411808,0.880051,0.886435
2,2022-01-12,0.378570,0.851364,0.903350
3,2022-01-13,0.399264,0.869483,0.883725
4,2022-01-14,0.334873,0.679499,0.788720


In [11]:
# #load the descriptions into textblob
# news_df_row = news_df.shape[0]
# news_df_col = news_df.shape[1]
# news_sentiment_subjectivity_list = []
# news_sentiment_vader_comp_list = []


# for i in range(news_df_row):
#     desc_blob = [TextBlob(desc) for desc in (news_df.iloc[i, 1:news_df_col])]
#     temp_subj_list = []    
#     #news_sentiment['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]             
#     news_sentiment_subjectivity_list.append([b.sentiment.subjectivity for b in desc_blob])
                 
#     news_sentiment_vader_comp_list.append([sia.polarity_scores(v)['compound'] for v in news_df.iloc[i, 1:news_df_col]])
    
# #     news_sentiment['vader_pos'] = [sia.polarity_scores(v)['pos'] for v in news_df.iloc[i, 1:news_df_col]]
# #     news_sentiment['vader_neu'] = [sia.polarity_scores(v)['neu'] for v in news_df.iloc[i, 1:news_df_col]]
# #     news_sentiment['vader_neg'] = [sia.polarity_scores(v)['neg'] for v in news_df.iloc[i, 1:news_df_col]]
    
# #     news_sentiment_list.append(news_sentiment)
# # news_sentiment_df = pd.concat(news_sentiment_list)
# # news_sentiment_df

# news_subjectivity_df = pd.DataFrame(news_sentiment_subjectivity_list)
# news_vader_compound_df = pd.DataFrame(news_sentiment_vader_comp_list)

In [12]:
#news_subjectivity_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.700758,0.105556,0.417316,0.455556,0.000000,0.850000,0.288889,0.440000,0.537500,0.000000,0.166667,0.300000,0.3425,0.914286,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.650000,0.450000,0.613690,0.438889,0.503788,0.259259,0.348485,0.638409,0.440198,0.541667,0.450000,0.424675,0.4500,0.606381,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,0.423571,0.193750,0.460000,0.264000,0.377273,0.492235,0.641667,0.372078,0.536458,0.453810,0.537500,0.408333,0.5000,0.000000,0.296703,0.557576,0.484127,0.339286,0.0,0.353571


In [13]:
#news_vader_compound_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.9419,0.6809,0.8733,0.9170,-0.2250,0.8918,0.0126,0.9718,0.9266,-0.2257,0.0401,0.5052,0.8855,0.9671,-0.0334,-0.0334,-0.0334,-0.0334,-0.0334,-0.0334
1,0.8730,0.9787,0.9615,0.7410,0.9593,0.9712,0.8641,0.9768,0.9098,0.9707,-0.5800,0.9051,-0.6090,0.9238,-0.0334,-0.0334,-0.0334,-0.0334,-0.0334,-0.0334
2,0.6963,0.6469,0.9437,0.9136,0.0175,0.8578,0.6690,0.9781,0.9908,-0.1944,0.9959,0.9218,0.8586,-0.0334,0.7039,0.9949,0.5116,0.8494,0.4266,0.5163


In [15]:
# def calculate_sim_score(d1, d2):
#     set_a, set_b = set(d1), set(d2)
#     return len(set_a and set_b) / len(set_a or set_b)

In [16]:
# news_df_row = news_df.shape[0]
# news_df_col = news_df.shape[1]
# news_sim_score = []

# for i in range(news_df_row):
#     sim_score_list = []
#     for j in range(0, news_df_col-1):
#         sim_scores = []
#         for k in range(0,(news_df_col-1)):
#             if j != k:
#                 #print(i,j,k)
#                 sim_scores.append(calculate_sim_score(news_df.iloc[i][j], news_df.iloc[i][k]))
#         sim_score_list.append(np.mean(sim_scores))
#     news_sim_score.append(sim_score_list)

In [17]:
# news_sim_score_df = pd.DataFrame(news_sim_score)
# news_sim_score_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.700585,0.735618,0.700585,0.552632,1.040747,0.575049,1.421053,0.863442,0.774069,0.717703,0.639098,0.700585,0.575049,0.521855,11.245614,11.245614,11.245614,11.245614,11.245614,11.245614
1,0.631579,0.644019,0.644019,0.929825,0.644019,0.670308,0.762598,0.670308,0.818182,0.670308,0.575496,0.859649,0.838433,0.656920,12.719298,12.719298,12.719298,12.719298,12.719298,12.719298
2,0.897773,0.897773,1.152760,0.955961,1.021739,0.845933,0.897773,0.879841,0.757550,0.976974,0.916409,0.862573,1.214575,16.421053,1.021739,0.757550,0.935789,0.829887,2.006579,1.021739


In [18]:
# news_weighted_subj_senti_df = news_subjectivity_df * news_vader_compound_df
#news_weighted_subj_senti_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.660044,0.071873,0.364442,0.417744,-0.000000,0.758030,0.003640,0.427592,0.498047,-0.000000,0.006683,0.151560,0.303284,0.884206,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.000000
1,0.567450,0.440415,0.590063,0.325217,0.483284,0.251793,0.301126,0.623598,0.400493,0.525796,-0.261000,0.384374,-0.274050,0.560175,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,-0.000000
2,0.294933,0.125337,0.434102,0.241190,0.006602,0.422239,0.429275,0.363929,0.531523,-0.088221,0.535296,0.376402,0.429300,-0.000000,0.208849,0.554732,0.247679,0.288189,0.0,0.182549


In [19]:
# news_weighted_simi_senti_df = news_sim_score_df * news_vader_compound_df
# news_weighted_simi_senti_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.659881,0.500882,0.611821,0.506763,-0.234168,0.512828,0.017905,0.839093,0.717253,-0.161986,0.025628,0.353935,0.509206,0.504686,-0.375604,-0.375604,-0.375604,-0.375604,-0.375604,-0.375604
1,0.551368,0.630302,0.619224,0.689000,0.617808,0.651003,0.658961,0.654757,0.744382,0.650668,-0.333788,0.778068,-0.510606,0.606863,-0.424825,-0.424825,-0.424825,-0.424825,-0.424825,-0.424825
2,0.625120,0.580770,1.087860,0.873366,0.017880,0.725641,0.600610,0.860573,0.750580,-0.189924,0.912651,0.795120,1.042834,-0.548463,0.719202,0.753686,0.478750,0.704906,0.856007,0.527524


In [20]:
# news_sentiment_df = []
# news_sentiment_df = news_df[['index']]
# news_sentiment_df['subj_score'] = news_weighted_subj_senti_df.mean(axis = 1, skipna = True)
# news_sentiment_df['simi_score'] = news_weighted_simi_senti_df.mean(axis = 1, skipna = True)
# news_sentiment_df['vader_score'] = news_vader_compound_df.mean(axis = 1, skipna = True)
# news_sentiment_df

C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

,index,subj_score,simi_score,vader_score
0,2021-08-30,0.227357,0.155505,0.398135
1,2021-08-31,0.245937,0.222953,0.482280
2,2021-09-01,0.279195,0.608735,0.663245
3,2021-09-02,0.166784,0.137560,0.432545
4,2021-09-03,0.134022,0.057392,0.352185
...,...,...,...,...
95,2022-01-10,0.300485,0.479747,0.635650
96,2022-01-11,0.331232,0.600482,0.711115
97,2022-01-12,0.300088,0.611451,0.723030
98,2022-01-13,0.331263,0.625200,0.708050


In [13]:
filepath_2 = 'Resources/sentiment_data/{}_test_sentiment.csv'.format(key)
filepath_2

'Resources/sentiment_data/tsla_test_sentiment.csv'

In [14]:
sentiment_df.to_csv(filepath_2)